In [16]:


import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms
from tqdm import tqdm
import numpy as np

In [17]:
device = torch.device('cuda:1')

In [18]:
!pip install pytorch-metric-learning

Defaulting to user installation because normal site-packages is not writeable


In [19]:
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(512,512)

/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [20]:
!pwd

/home/ubuntu


In [21]:
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='NVIDIA GeForce RTX 3080 Ti', major=8, minor=6, total_memory=12053MB, multi_processor_count=80)

In [22]:
model_state_dict = torch.load('./lialib/colab/rn18_200epochs.pth',map_location=device)

In [23]:
#model.load_state_dict(model_state_dict['rn18'])


In [24]:
class_order = model_state_dict['class_order']

In [25]:
class_order

tensor([89,  0, 70, 84, 99, 65, 63, 60, 67, 61, 80, 87, 92,  9, 37, 35,  8, 93,
        21, 57, 53, 25, 43, 68, 88, 10, 30, 46, 39, 14, 22, 82, 83, 20, 96, 58,
         7,  5, 78, 38, 81, 71, 56, 32, 72, 31, 34, 45, 16, 33, 13, 29, 41, 49,
        86, 76, 64, 15, 94, 26, 55, 79, 52,  2, 91, 11, 17, 44, 74,  4, 24, 73,
        77, 98, 28, 95, 27, 47, 48, 42, 69, 40, 18, 36, 97,  3,  6, 85, 12,  1,
        66, 51, 62, 54, 50, 90, 75, 19, 23, 59], device='cuda:1')

In [26]:
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
])

In [27]:
ds_train = datasets.CIFAR100(root='./data', train=True, download=True, transform=preprocess)
ds_test  = datasets.CIFAR100(root='./data', train=False, download=True, transform=preprocess)

Files already downloaded and verified
Files already downloaded and verified


In [28]:
def instances_from_classes(dataset, class_order):
    subset = []
    for i in range(len(dataset)):
        _, label = dataset[i]
        if label in class_order:
            subset.append(i)
    return subset


In [29]:
set_train = instances_from_classes(ds_train,class_order[:50])
set_test  = np.array(instances_from_classes(ds_test, class_order[:50]))

In [30]:
set_test.shape

(5000,)

In [31]:
sub_sample_train = sorted(np.random.permutation(set_train)[:300])
sub_sample_test  = sorted(np.random.permutation(set_test)[:300])

In [32]:
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=64, sampler=torch.utils.data.SubsetRandomSampler(set_train))
dl_test = torch.utils.data.DataLoader(ds_test,  batch_size=64, sampler=torch.utils.data.SubsetRandomSampler(set_test))
dl_sub_train = torch.utils.data.DataLoader(ds_train,  batch_size=64, sampler=torch.utils.data.SubsetRandomSampler(sub_sample_train))
dl_sub_test = torch.utils.data.DataLoader(ds_test,  batch_size=64, sampler=torch.utils.data.SubsetRandomSampler(sub_sample_test))

In [33]:
x,y = next(iter(dl_train))

In [34]:
y.shape

torch.Size([64])

In [35]:
y

tensor([71,  7,  7, 84, 61, 93, 58, 53, 87, 53, 22, 21, 56, 56, 65,  8, 68, 16,
        45, 96, 43, 89, 82, 65,  8, 82,  9, 92, 88, 63, 71, 60, 60, 45, 25, 34,
        46, 84, 20, 60, 93, 34, 33, 57, 38, 82, 80, 68, 78,  8, 71, 93,  7, 14,
        80, 80, 92, 38,  8, 43, 81, 16, 30, 39])

In [36]:
from pytorch_metric_learning import losses
loss_func = losses.ArcFaceLoss(num_classes=100,embedding_size=512)


In [37]:
!pip install wandb

Defaulting to user installation because normal site-packages is not writeable


In [38]:
import wandb

In [39]:
wandb.init(
    # set the wandb project where this run will be logged
    project="clip+dmil",
)

wandb: Currently logged in as: edson-matsubara (lablia). Use `wandb login --relogin` to force relogin


In [40]:

opt = optim.Adam(model.parameters(),lr=0.001)

In [42]:
scheduler = optim.lr_scheduler.MultiStepLR(opt,milestones=[45,90],gamma=0.1)

In [43]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [44]:
emb = model(x.to(device))

In [45]:
y

tensor([71,  7,  7, 84, 61, 93, 58, 53, 87, 53, 22, 21, 56, 56, 65,  8, 68, 16,
        45, 96, 43, 89, 82, 65,  8, 82,  9, 92, 88, 63, 71, 60, 60, 45, 25, 34,
        46, 84, 20, 60, 93, 34, 33, 57, 38, 82, 80, 68, 78,  8, 71, 93,  7, 14,
        80, 80, 92, 38,  8, 43, 81, 16, 30, 39])

In [46]:
emb.shape

torch.Size([64, 512])

In [47]:
y.shape

torch.Size([64])

In [50]:
loss = loss_func(emb, y)

In [51]:
def get_embeddings(lmodel,dl):
    lmodel.to(device)
    loop = tqdm(dl)
    lpred = []
    ly = []
    with torch.no_grad():
        for x,y in loop:
            x = x.to(device)
            y = y.to(device)
            pred = lmodel(x)
            lpred = lpred+pred.tolist()
            ly    = ly + y.tolist()
    return lpred,ly

In [52]:
import sklearn.neighbors as neighbors

In [58]:
from sklearn.metrics import classification_report

In [59]:
def pred_knn_model(model,dl_local_train,dl_local_test):
    train_emb,train_y = get_embeddings(model,dl_local_train)
    test_emb,test_y   = get_embeddings(model,dl_local_test)
    train_emb  = np.array(train_emb)
    train_y    = np.array(train_y)
    test_emb   = np.array(test_emb)
    test_y     = np.array(test_y)
    clf = neighbors.KNeighborsClassifier(n_neighbors=1,weights='distance')
    clf.fit(train_emb,train_y.T)
    pred = clf.predict(test_emb)
    return pred,test_y


In [62]:
pred,test_y = pred_knn_model(model,dl_sub_train,dl_sub_test)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 12.14it/s]
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

In [65]:
pred.shape,test_y.T.shape

((300,), (300, 1))

In [67]:
print(classification_report(test_y.T,pred))

              precision    recall  f1-score   support

           0       0.67      0.40      0.50         5
           5       0.17      0.12      0.14         8
           7       0.00      0.00      0.00         7
           8       0.00      0.00      0.00         6
           9       0.25      0.20      0.22         5
          10       0.00      0.00      0.00         5
          14       0.00      0.00      0.00         2
          16       0.00      0.00      0.00        10
          20       0.06      0.25      0.10         4
          21       0.20      0.14      0.17         7
          22       0.00      0.00      0.00         9
          25       0.00      0.00      0.00         5
          30       0.56      0.71      0.63         7
          31       0.25      0.10      0.14        10
          32       0.00      0.00      0.00         3
          33       0.00      0.00      0.00         7
          34       0.17      0.20      0.18         5
          35       0.06    

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [68]:
model.to(device)
loss_func.to(device)
loss_train = []
for epoch in range(100):
    model.train()
    loop = tqdm(dl_train)
    lloss = []
    for data, labels in loop:
        data = data.to(device)
        labels = labels.to(device)
        opt.zero_grad()
        embeddings = model(data)
        loss = loss_func(embeddings, labels)
        loss.backward()
        lloss.append(loss.item())
        opt.step()
    scheduler.step()
    if epoch %10 == 0:
        pred,test_y = pred_knn_model(model,dl_sub_train,dl_sub_test)
        print(classification_report(test_y.T,pred))
    loss_train.append(np.mean(lloss))
    print(f"loss {loss_train[-1]}")
    wandb.log({'loss_train':loss_train[-1]})

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 11.37it/s]
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           5       0.25      0.12      0.17         8
           7       0.00      0.00      0.00         7
           8       0.06      0.17      0.08         6
           9       1.00      0.40      0.57         5
          10       0.25      0.20      0.22         5
          14       0.00      0.00      0.00         2
          16       0.14      0.10      0.12        10
          20       0.00      0.00      0.00         4
          21       0.25      0.29      0.27         7
          22       0.00      0.00      0.00         9
          25       0.50      0.20      0.29         5
          30       0.33      0.29      0.31         7
          31       0.00      0.00      0.00        10
          32       0.00      0.00      0.00         3
          33       0.12      0.14      0.13         7
          34       0.14      0.40      0.21         5
          35       0.00    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 391/391 [00:52<00:00,  7.52it/s]


31.019898787788723


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 391/391 [00:51<00:00,  7.63it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 391/391 [00:51<00:00,  7.66it/s]


0.009982987767112346


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 391/391 [00:49<00:00,  7.89it/s]


0.012731620468387758


 16%|████████████████████████▎                                                                                                                                | 62/391 [00:08<00:42,  7.70it/s]

In [ ]:
torch

<module 'torch' from '/home/ubuntu/.local/lib/python3.10/site-packages/torch/__init__.py'>

In [ ]:
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='NVIDIA GeForce RTX 3060', major=8, minor=6, total_memory=12044MB, multi_processor_count=28)

In [ ]:
torch.save({'rn18':model.state_dict(),'class_order':class_order,'loss_train':loss_train},'rn18_pass_100epochs.pth')